# 学习目标：
- 掌握Qwen模型基本特性
- 掌握Baichuan-7B基本特性
- 掌握NLP四种范式，并知道

# 1、Qwen模型
概述：Qwen大语言模型是是有阿里巴巴训练并开源的一系列大语言模型，最早开始2023年8月开源70亿参数规模，随后几个月开始陆续开源其他模型，最低参数18亿，最高参数720亿

训练目标：在训练目标上，Qwen的核心训练目标是语言模型，即根据已有的上下文去预测下一个词
模型结构：
- 分组查询注意力：通过减少键值（KV）缓存的作用，提升了模型的推理效率
- 激活函数：使用SwiGLU激活函数，增强了模型的非线性表达能力
- 位置编码：采用了旋转编码RoPE，提升了模型对长序列的处理能力
- 层归一化（Layer Normalization）：使用了RMSNorm，确保了训练的稳定性

模型配置：

| 配置       | 数据                    |
| ---------- | ----------------------- |
| 参数       | 7B                      |
| 隐藏层维度 | 3584                    |
| 层数       | 28                      |
| 注意力头数 | 28个query、4个key-value |
| 训练数据   | 1.2T                    |
| 词表大小   | 151936                  |
| 最大长度   | 32768                   |


大规模叶柄，日本在美国

# 2、Baichuan-7B模型
概述：Baichuan-7B是由百川智能于2023年6月发布的一个开发且可商用的大型预训练，支持中英文双语，是在约1.2万亿哥token上训练70亿参数模型

训练目标：
- Baichuan-7B的训练目标也是语言模型，即根据已有的上文去预测下一个词
- 关于tokenizer使用了BPE分词算法作为tokenizer，词表大小64000
- 关于数据，原始数据包括开源的中英文数据和自行抓取的中文互联网数据，以及部分高质量知识性数据

模型结构：
- 和LLaMA一样的模型设计，也是Decoder-Only架构，但是结合前人的工作做了一些改进
    - Pre-Normalization：为了提高训练的稳定性，没有使用传统的Post-Layer-Norm 而是使用了Pre-Layer-Norm并同时使用RMSNorm归一化函数（RMS Norm的主要区别在于去掉了减去均值部分，简化了LayerNorm的计算，可以减少约7%～64%计算时间）
    - 激活函数：使用SwiGLU激活函数
    - 位置编码：采用旋转位置编码RoPE

模型配置：

| 配置       | 数据    |
|:---------|:------|
| 参数       | 7B    |
| 隐藏层维度    | 4096  |
| 层数       | 32    |
| 注意力头数    | 32    |
| 训练数据     | 1.2T  |
| 词表大小     | 64000 |
| 最大长度     | 4096  |



# 1、NLP四种范式分别是？
- 第一范式：基于“传统机器学习模型”的范式
- 第二范式：基于“深度学习模型”的范式
- 第三范式：基于“预训练模型+Fine+Tuning”的范式
- 第四范式：基于“预训练模型+Prompt+预测”的范式


# Prompt-Tuning（提示微调）
Prompt-Tuning执行步骤：
- 构建模版（Template Construction）：通过人工定义、自动搜索、文本生成等方法，生成与给定句子相关的[MASK]标记模版，例如：It was [MASK]，并拼接到原始的文本中获取Prompt-Tuning的输入：[CLS] I like the Disney films very much, [SEP] It was [MASK]. [SEP],将其喂给BERT模型中，并重复预训练好的MLM分类器（在Huggingface中为BertForMaskedLM），即可以得到[MAKS]预测结果中各个token分布概率
- 标签词映射（Label Word Verbalizer）：因为[MASK]部分我们只对部分敏感词感兴趣，因此需要建立一个映射关系，例如：如果[MASK]预测词是“great”则认为是positive类，如果是“terrible”则认为是negative类
- 训练：根据Verbalizer则可以获得制定的label word的预测概率分布，采用交叉信息熵进行训练，此时因为只对预训练好的MLM head进行微调，所以避免了过拟合问题



# PET模型
PET模型提出来两个很重要的组件：
- Pattern（Template）：记做T上文提到了Template，其额外添加带有[MASK]标记的短文本，通常一个文本只有一个Pattern（因为我们希望只有一个让模型预测的[MASK]标记），有不同的任务，不同的样本可能会有其更加适合的Pattern，因此如何构建Pattern是Pormpt-Tuning的研究点之一
- Verbalizer：记做V即标签词的映射，对于具体的分类任务，需要选择指定的标签词（label word）例如情感分析中，我们期望Verbalizer可能是（Positive和Negative是类标签），同样不同的任务有其相应的label word，但是需要注意的是Verbalizer的构建需要取决于对应的Pattern，因此如何构建Verbalizer是另一个挑战


# Hard Prompt & Soft Prompt （针对Prompt调优两种方法：Hard Prompt和soft Prompt）
- Hard Prompt (离散提示)：是一种固定的提示模板，通过将特定的关键词或短语(真实的文本字符串)直接嵌入到文本中，引导模型生成符合要求的文本。这种提示方法的特点在于，提示模板是固定的，不能根据不同的任务和需求进行调整。
- Soft Prompt (连续提示) ：是指通过给模型输入一个可参数化的提示模板，从而引导模型生成符合特定要求的文本。这种提示方法的特点在于，提示模板中的参数可以根据具体任务和需求进行调整，以达到最佳的生成效果。

# P-tuning（NLU任务）

P-Tuning是2022年清华大学在论文呢（FPT Understands Too）中提出的微调方法，该方法提出将Prompt转换为可以学习的Embedding层，只在考虑到直接对Embedding参数进行优化

![P-Tuning](./img/P-Tuning.png)

基本原理：
P-Tuning固定LLM参数，利用多层感知机（MLP）和LSTM对Prompt进行编码，编码之后的响亮进行拼接之后正常输入LLM注意，训练之后只保留Prompt编码之后的向量即可，无需保留编码器，

P-Tuning V2是升级版本，主要解决P-Tuning在V1小参数模型上表现差的问题
核心思想：在模型的每一层都应用联系的prompts，并对prompts参数进行更新优化，该方法也针对NLU任务优化和适配的


# Prefix-Tuning
定义：2021年论文《Prefix-Tuning: Optimizing Continuous Prompts for Generation》中提出了 Prefix Tuning 方法，该方法是在输入 token 之前构造一段任务相关的 virtual tokens 作为 Prefix，然后训练的时候只更新 Prefix 部分的参数，而 Transformer 中的其他部分参数固定.

![Prefix-Tuning](./img/Prefix-Tuning.png)

比较P-Tuning相同：
- Prefix-Tuning 是将额外的embedding加在开头，看起来更像模仿Instruction指令，而P-Tuning 位置不固定.
- Prefix-Tuning 通过在每个层都添加可训练参数，通过MLP初始化，而P-Tuning只在输入的时候加入embedding, 并通过LSTM+MLP初始化.


# Adapter-Tuning
不同于Prefix-Tuning这类输入可添加训练prompt参数，Adapter-Tuning则是在训练模型内部网络层之间添加新的网络层或模块来适配下游任务，当模型训练时，固定在原来的预训练模型参数不变，只是新增的Adapter结构进行微调

![Adapter-Tuning](./img/Adapter-Tuning.png)

内部Adapter-Tuning构造：
> 首先一个down-project层将高纬度特征映射到低纬度，然后经过一个非线性层，再用一个up-project结构将低维特征映射回来原本的高纬特征，同时也涉及一个skip-connection结构，确保在最差的时候能够退化称原本的identity（类似残差结构）
 
# LoRA微调
概念：低秩矩阵（low-Rank Adaption）是一种参数高效的微调方法，其核心思想是对大模型的权重矩阵进行隐式的低秩转换，也就是通过一个较低纬度的表示来近似一个高纬度矩阵的数据集


LoRA产生原因：

Adapter-Tuning方法PLM 基础上添加适配器层会引入额外的计算，带来推理延迟问题；而 Prefix Tuning 方法难以优化，其性能随可训练参数规模非单调变化，更根本的是，为前缀保留部分序列长度必然会减少用于处理下游任务的序列长度. 因此微软推出了LoRA方法.

![LoRA](./img/LoRA.png)


基本原理：LoRA技术冻结预训练模型的权重，并在每个Transformer块中注入可训练层（称为低秩分解矩阵），即在模型的Linear层的旁边增加一个“旁支”A和B。其中，A将数据从d维降到r维，这个r是LoRA的秩，是一个重要的超参数；B将数据从r维升到d维，B部分的参数初始为0。模型训练结束后，需要将A+B部分的参数与原大模型的参数合并在一起使用。

In [1]:

from transformers import BertTokenizer, BertForMaskedLM
import torch

# 加载预训练的 BERT 模型和 tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

# 设计 prompt（提示）
text = "Hamlet was written by [MASK]."

# 对输入进行编码
input_ids = tokenizer.encode(text, return_tensors="pt")

# 找出 [MASK] 的位置索引
mask_token_indices = torch.where(input_ids == tokenizer.mask_token_id)[1]

# 模型推理
with torch.no_grad():
    outputs = model(input_ids)
    logits = outputs.logits

# 获取 [MASK] 位置的预测结果
mask_logits = logits[0, mask_token_indices, :]

# 获取概率最高的 top 5 个词
top_tokens = torch.topk(mask_logits, 5, dim=1).indices.tolist()[0]

# 输出预测结果
print("Top 5 predictions for [MASK]:")
for token in top_tokens:
    print(tokenizer.decode([token]))

/Users/BrownSugar/miniconda3/envs/AI-Lesson/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: Can't load tokenizer for 'bert-base-uncased'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'bert-base-uncased' is the correct path to a directory containing all relevant files for a BertTokenizer tokenizer.